In [15]:
import pandas as pd
import nltk
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from xin_util.Euclidean_vectors import euclidean_df_array_faster
import seaborn as sns
from xin_util.Rank_top_n import find_top_n_faster
from sklearn.ensemble import RandomForestClassifier
import ipywidgets as widgets
from ipywidgets import interact, Layout

from ipywidgets import interact
import numpy as np
import re
from collections import defaultdict
import matplotlib.pyplot as plt
from xin_util.CreateDIYdictFromDataFrame import CreateDIYdictFromDataFrame
import google.cloud.bigquery as bigquery
import math
import os
import copy
from scipy.stats import boxcox
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tqdm import tqdm
tqdm.pandas()
import google.cloud.bigquery as bigquery
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning) 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
sns.set_style("white")
import psutil
print(f'current memory usage {psutil.virtual_memory().percent}%')


current memory usage 14.1%


In [5]:
DATE_TO_QUERY= "2020-12-01"

client = bigquery.Client(project='geotab-bi')
SQL = '''
            SELECT * FROM (select * from geotab-bi.SafetyPrototype.Step_11_Battery_Profile WHERE DATE(_PARTITIONTIME) = "{date_val}")
                     JOIN (select * from geotab-bi.SafetyPrototype.Step_11_Pressure_Profile WHERE DATE(_PARTITIONTIME) = "{date_val}")
                    using (HardwareId)
                     join (select * from geotab-bi.SafetyPrototype.Step_11_Speed_Profile WHERE DATE(_PARTITIONTIME) = "{date_val}")
                    using (HardwareId)
                     join (select * from geotab-bi.SafetyPrototype.Step_11_Temperature_Profile WHERE DATE(_PARTITIONTIME) = "{date_val}")
                    using (HardwareId)
                     join (select * from geotab-bi.SafetyPrototype.Step_11_Snow_Profile WHERE DATE(_PARTITIONTIME) = "{date_val}")
                    using (HardwareId)
                     join (select * from geotab-bi.SafetyPrototype.Step_11_Precipitation_Profile WHERE DATE(_PARTITIONTIME) = "{date_val}")
                    using (HardwareId)
                     join (select * from geotab-bi.SafetyPrototype.Step_10_Features WHERE DATE(_PARTITIONTIME) = "{date_val}")
                    using (HardwareId)
            '''.format(date_val=DATE_TO_QUERY)
vehicle_feature = client.query(SQL).to_dataframe()
vehicle_feature.head()


,HardwareId,battery_profile_1st,battery_profile_2nd,battery_profile_3rd,battery_profile_4th,pressure_profile_1st,pressure_profile_2nd,pressure_profile_3rd,pressure_profile_4th,speed_profile_1st,speed_profile_2nd,speed_profile_3rd,speed_profile_4th,temperature_profile_1st,temperature_profile_2nd,temperature_profile_3rd,temperature_profile_4th,snow_profile,precipitation_profile,Vin,Make,Model,Year,VehicleType,WeightClass,HoursInterval,DistanceInterval,VocationId,HorsepowerInterval,DisplacementInterval,speed_0,speed_10,speed_20,speed_30,speed_40,speed_50,speed_60,speed_70,speed_80,speed_90,speed_100,speed_110,Temperature_minus40,Temperature_minus30,Temperature_minus20,Temperature_minus10,Temperature_0,Temperature_10,Temperature_20,Temperature_30,Temperature_40,Temperature_50,Temperature_above50,Precipitation_below50,Precipitation_above50,Snow_below50,Snow_above50,Pressure_85000,Pressure_98000,Pressure_99000,Pressure_100000,Pressure_110000,Pressure_above110000,geo3,geo2,battery_0,battery_10,battery_20,battery_30,battery_40,battery_50,battery_60,battery_70,battery_80,battery_90,OilChangeInterval
0,550227075,battery_0,battery_10,battery_20,battery_30,Pressure_110000,Pressure_100000,Pressure_98000,Pressure_85000,Ratio_0,Ratio_80,Ratio_70,Ratio_90,Temperature_30,Temperature_40,Temperature_20,Temperature_10,Snow_above95,Precipitation_above85,3HAEUMMR2ML550606,International,MV,2021,Truck,Class 8 (33001 - Over lbs),0,0,LongDistance9To5,0,6.0,0.144955,0.037313,0.046395,0.045337,0.057708,0.081630,0.069350,0.127672,0.141279,0.125850,0.113665,0.008845,NaN,NaN,NaN,NaN,NaN,0.004738,0.036658,0.667402,0.289164,0.002037,NaN,1.000000,NaN,1.000000,NaN,NaN,0.068687,NaN,0.312870,0.618443,NaN,9s7,9s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999999
1,550775928,battery_0,battery_10,battery_20,battery_30,Pressure_98000,Pressure_85000,Pressure_99000,Pressure_100000,Ratio_0,Ratio_10,Ratio_20,Ratio_50,Temperature_10,Temperature_20,Temperature_0,Temperature_30,Snow_above95,Precipitation_above85,1FTER1FHXKLA24435,Ford,Ranger,2019,Truck,Class E (6001 - 7000 lbs),0,0,LongStop9To5,240,0.0,0.612352,0.069553,0.062374,0.039158,0.032863,0.047020,0.033241,0.037254,0.030048,0.027458,0.008456,0.000223,NaN,NaN,NaN,NaN,0.113182,0.507276,0.301692,0.077850,NaN,NaN,NaN,0.974943,0.025057,0.963585,0.036415,NaN,1.000000,NaN,NaN,NaN,NaN,dnt,dn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999999
2,550880391,battery_0,battery_10,battery_20,battery_30,Pressure_99000,Pressure_98000,Pressure_100000,Pressure_110000,Ratio_110,Ratio_0,Ratio_100,Ratio_90,Temperature_10,Temperature_20,Temperature_30,Temperature_0,Snow_above95,Precipitation_above85,2HSCKSCR97C429337,International,9900ix,2007,Truck,Class 8 (33001 - Over lbs),90,7000,LongDistance9To5,500,15.0,0.295455,0.023907,0.012947,0.009626,0.014513,0.021235,0.020910,0.028223,0.035882,0.050741,0.106496,0.380063,NaN,NaN,NaN,NaN,0.018162,0.620868,0.252102,0.108857,0.000012,NaN,NaN,0.984459,0.015541,0.953568,0.046432,NaN,0.432244,0.490748,0.066263,0.010746,NaN,dpq,dp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999999
3,551181177,battery_0,battery_10,battery_20,battery_30,Pressure_98000,Pressure_99000,Pressure_85000,Pressure_100000,Ratio_0,Ratio_110,Ratio_100,Ratio_90,Temperature_30,Temperature_10,Temperature_20,Temperature_0,Snow_above95,Precipitation_above85,3AKJGLD64GSGY7220,Freightliner,Cascadia 125 Sleepercab,2016,Truck,Class 8 (33001 - Over lbs),0,0,LongDistance9To5,500,7.3,0.517705,0.018128,0.013470,0.009302,0.011419,0.012642,0.010176,0.014199,0.014614,0.021241,0.052474,0.304630,NaN,NaN,NaN,NaN,0.000581,0.351190,0.275004,0.373225,NaN,NaN,NaN,1.000000,NaN,1.000000,NaN,NaN,0.996431,0.003569,NaN,NaN,NaN,9yf,9y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999999
4,551714588,battery_0,battery_10,battery_20,battery_30,Pressure_99000,Pressure_100000,Pressure_85000,Pressure_98000,Ratio_0,Ratio_70,Ratio_80,Ratio_100,Temperature_10,Temperature_20,Temperature_30,Temperature_0,Snow_above95,Precipitation_above85,3HAMMMML2JL707098,International,DuraStar 4300,2018

In [7]:
RAW_FEATURES= [ 'VehicleType',
                'WeightClass',
                'HoursInterval',
                'DistanceInterval',
                'VocationId',
                'HorsepowerInterval',
                'DisplacementInterval',
                'geo2',
                'OilChangeInterval',
                'speed_profile_1st',
                'speed_profile_2nd',
                'speed_profile_3rd',
                'speed_profile_4th',
                'temperature_profile_1st',
                'temperature_profile_2nd',
                'temperature_profile_3rd',
                'temperature_profile_4th',
                'precipitation_profile',
                'snow_profile',
                'pressure_profile_1st',
                'pressure_profile_2nd',
                'pressure_profile_3rd',
                'pressure_profile_4th',
                'battery_profile_1st',
                'battery_profile_2nd',
                'battery_profile_3rd',
                'battery_profile_4th' ]

METADATA= [ 'HardwareId', 'Vin', 'Make', 'Model' ]

def preprocess_data(dataset: pd.DataFrame):

    raw_features_col = RAW_FEATURES

    metadata_col = METADATA

    metadata = dataset[metadata_col]

    features = pd.get_dummies(dataset[raw_features_col], prefix_sep='_',
                              drop_first=False,
                              columns=raw_features_col)

    return features.values, features, metadata.values

In [8]:
X, features, metadata = preprocess_data(vehicle_feature)
X_df = pd.DataFrame(X)

In [6]:
from numba import cuda, float32

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [12]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

In [13]:
matmul(X[0:10000, ], X[0:10000, ].T)

ValueError: 
Kernel launch configuration was not specified. Use the syntax:

kernel_function[blockspergrid, threadsperblock](arg0, arg1, ..., argn)

See https://numba.pydata.org/numba-doc/latest/cuda/kernels.html#kernel-invocation for help.



In [14]:
@cuda.jit
def increment_a_2D_array(an_array):
    x, y = cuda.grid(2)
    if x < an_array.shape[0] and y < an_array.shape[1]:
       an_array[x, y] += 1

In [19]:
an_array = np.array([[1,2,3,4,5,6], [1,2,3,4,5,6]])

threadsperblock = (16, 16)
blockspergrid_x = math.ceil(an_array.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(an_array.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)
increment_a_2D_array[blockspergrid, threadsperblock](an_array)